# Web Scraping

In [73]:
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

my_url = 'https://www.imdb.com/list/ls068010962/'
#opening up connection, downloading the page
uClient = uReq(my_url)
bol_celeb = uClient.read()
uClient.close()

#html parsing
bol_soup = soup(bol_celeb,'html.parser')

#grabs each star
containers =  bol_soup.findAll('div',{'class':'lister-item mode-detail'})

filename = 'bollywood.csv'
f= open(filename,'w')
headers = 'actor_image,actor_name,movie,about\n'
f.write(headers)

pic_name = []
actor_name = []
actor_desc = []
for container,img in zip(containers,bol_soup.findAll('img')):
    name = container.h3.text
    movie = container.p.text.replace('Actor |',"")
    trait =container.find('p',{'class':''}).text
    image = img.get('src')
    image_name = img.get('alt')
    full_name = str(image_name) + ".jpg"
    pic_name.append(full_name)
    actor_name.append(name)
    actor_desc.append(trait)
    urllib.request.urlretrieve(image, full_name)

# Uploading to SQLite database

In [76]:
import sqlite3

def convertToBinaryData(filename):
    #Convert digital data to binary format
    with open(filename, 'rb') as file:
        blobData = file.read()
    return blobData

def insertBLOB(index, name, photo, description):
    try:
        sqliteConnection = sqlite3.connect('SQLite_Python.db')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")
        sqlite_insert_blob_query = """ INSERT INTO new_actor
                                  (index, name, photo, description) VALUES (?, ?, ?, ?)"""

        actPhoto = convertToBinaryData(photo)
        # Convert data into tuple format
        data_tuple = (index, name, actPhoto, description)
        cursor.execute(sqlite_insert_blob_query, data_tuple)
        sqliteConnection.commit()
        print("Image inserted successfully as a BLOB into a table")
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert blob data into sqlite table", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("the sqlite connection is closed")

In [85]:
id = range(1,101)
for a,p,d,i in zip(actor_name,pic_name,actor_desc,range(1,101)):
    path = 'C:\\Users\\Manoj\\'+p
    insertBLOB(1, a, path, d)